In [30]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier


In [2]:
df = pd.read_csv('/content/Iris.csv')

In [3]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


# Bagging using 3 models

In [4]:
df = df[['SepalWidthCm','PetalWidthCm','Species']]

In [5]:
df['Species'].value_counts()

,count
Species,
Iris-setosa,50
Iris-versicolor,50
Iris-virginica,50


In [6]:
le = LabelEncoder()
df['Species']=le.fit_transform(df['Species'])

In [7]:
df =  df.sample(100)
df_train = df.iloc[:60,:].sample(50)
df_val = df.iloc[60:80,:].sample(5)
df_test = df.iloc[80:,:].sample(5)

In [8]:
df_train.head()

,SepalWidthCm,PetalWidthCm,Species
73,2.8,1.2,1
17,3.5,0.3,0
49,3.3,0.2,0
1,3.0,0.2,0
116,3.0,1.8,2


# Creating models

# Model 1

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import accuracy_score

In [10]:
df1 = df_train.sample(8,replace=True)

In [11]:
dt1 = DecisionTreeClassifier()
dt1.fit(df1.iloc[:,:-1].values,df1.iloc[:,-1].values)

DecisionTreeClassifier()

In [12]:
dt1.score(df_val.iloc[:,:-1].values,df_val.iloc[:,-1].values)

0.6

# Model 2

In [13]:
df2 = df_train.sample(8,replace=True)

In [14]:
dt2 = DecisionTreeClassifier()
dt2.fit(df2.iloc[:,:-1].values,df2.iloc[:,-1].values)

DecisionTreeClassifier()

In [15]:
dt2.score(df_val.iloc[:,:-1].values,df_val.iloc[:,-1].values)

0.8

# Model 3

In [16]:
df3 = df_train.sample(8,replace=True)

In [17]:
dt3 = DecisionTreeClassifier()
dt3.fit(df3.iloc[:,:-1].values,df3.iloc[:,-1].values)

DecisionTreeClassifier()

In [18]:
dt3.score(df_val.iloc[:,:-1].values,df_val.iloc[:,-1].values)

0.6

In [19]:
df.head()

,SepalWidthCm,PetalWidthCm,Species
53,2.3,1.3,1
51,3.2,1.5,1
64,2.9,1.3,1
44,3.8,0.4,0
135,3.0,2.3,2


In [20]:
np.array([1,2]).reshape(1,2)

array([[1, 2]])

In [21]:
print(dt1.predict(np.array([3.9,0.4]).reshape(1, 2)))
print(dt2.predict(np.array([3.9,0.4]).reshape(1, 2)))
print(dt3.predict(np.array([3.9,0.4]).reshape(1, -2)))

[0]
[0]
[0]


# Bagging library


In [26]:
from sklearn.model_selection import  train_test_split,GridSearchCV


In [23]:
x = df.iloc[:,:-1]
y = df.iloc[:,-1]
X_train,X_test,y_train,y_test = train_test_split(x,y,random_state=42)

In [24]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier()
bc.fit(X_train,y_train)

BaggingClassifier()

In [27]:
accuracy_score(y_test,bc.predict(X_test))

0.96

# Decision tree result

In [28]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train,y_train)

DecisionTreeClassifier()

In [29]:
accuracy_score(y_test,decision_tree.predict(X_test))

0.92

# finding best parameters using grid search cv

In [32]:
params = {
    'estimator' : [None,DecisionTreeClassifier(),KNeighborsClassifier()],
    'n_estimators' : [10,100,500,1000],
    'max_samples' : [0.25,0.5,1.0],
    'max_features' : [0.25,0.5,1.0],
    'bootstrap_features':[True,False],
    'bootstrap' : [True,False]
}

grid_search = GridSearchCV(BaggingClassifier(),param_grid=params,cv=5,scoring='accuracy')

In [33]:
grid_search.fit(X_train,y_train)


GridSearchCV(cv=5, estimator=BaggingClassifier(),
             param_grid={'bootstrap': [True, False],
                         'bootstrap_features': [True, False],
                         'estimator': [None, DecisionTreeClassifier(),
                                       KNeighborsClassifier()],
                         'max_features': [0.25, 0.5, 1.0],
                         'max_samples': [0.25, 0.5, 1.0],
                         'n_estimators': [10, 100, 500, 1000]},
             scoring='accuracy')

In [34]:
grid_search.best_score_

np.float64(0.9333333333333333)

In [35]:
grid_search.best_estimator_.score(X_test,y_test)

1.0